패키지 설치

In [0]:
!pip install mxnet-cu100
!pip install gluonnlp
!pip install gluoncv

     |████████████████████████████████| 540.1MB 46kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 245kB 1.4MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.8.0-cp36-none-any.whl size=292704 sha256=92ad06adc4590f1df92c1c48804404d71da556919e8a16fa5aeeaf26f27681cb
  Stored in directory: /root/.cache/pip/wheels/28/ff/33/d73801f242fb93c02f2076f81232fcb9a29305480cc42c5454
Successfully built gluonnlp
     |████████████████████████████████| 348kB 1.4MB/s 


In [0]:
!nvidia-smi

Fri Aug  9 04:06:40 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    16W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
from mxnet import gluon

import warnings
warnings.simplefilter('ignore')

import mxnet as mx
from mxnet import nd
import gluonnlp as nlp
import re

Vocab 생성

In [0]:
text = """
지난달 일본 수출 규제 조치로 촉발된 일본제품 불매운동 여파로 유니클로 무인양품 등 일본 브랜드의 모바일 앱 사용자가 급격히 감소했다는 조사 결과가 나왔다
"""

In [0]:
def simple_tokenize(source_str, token_delim=' ', seq_delim='\n'):
    return filter(None, re.split(token_delim + '|' + seq_delim, source_str))

counter = nlp.data.count_tokens(simple_tokenize(text))

In [0]:
vocab = nlp.Vocab(counter)

In [0]:
for word in vocab.idx_to_token:
    print(word)

<unk>
<pad>
<bos>
<eos>
일본
감소했다는
결과가
규제
급격히
나왔다
등
모바일
무인양품
불매운동
브랜드의
사용자가
수출
앱
여파로
유니클로
일본제품
조사
조치로
지난달
촉발된


이전 학습 데이터 기준 Word Embedding 생성

In [0]:
source_list = nlp.embedding.list_sources('fasttext')

In [0]:
for s in source_list:
    if 'ko' in s:
        print(s)

wiki.koi
wiki.ko
cc.ko.300


In [0]:
fasttext_simple = nlp.embedding.create('fasttext', source='wiki.ko')

In [0]:
vocab.set_embedding(fasttext_simple)

In [0]:
print('Vocabulary has length', len(vocab))
print(vocab.idx_to_token)

Vocabulary has length 25
['<unk>', '<pad>', '<bos>', '<eos>', '일본', '감소했다는', '결과가', '규제', '급격히', '나왔다', '등', '모바일', '무인양품', '불매운동', '브랜드의', '사용자가', '수출', '앱', '여파로', '유니클로', '일본제품', '조사', '조치로', '지난달', '촉발된']


In [0]:
vocab.embedding['수출']


[ 4.2638e-01 -3.4225e-01  6.3850e-02  1.8274e-01 -6.9525e-02 -8.3702e-01
  1.9475e-01 -1.5824e-02 -1.7478e-04  4.9751e-02 -3.3179e-03  2.0388e-01
 -5.5108e-02  4.6147e-01 -4.2385e-01  2.5457e-01 -1.4857e-01  1.8358e-01
 -7.3676e-02  1.9369e-01  2.7336e-01 -3.5977e-01 -3.1542e-02 -3.2799e-01
 -1.0815e-01  8.5354e-02 -1.4569e-01 -4.0252e-01  9.3765e-03  5.4110e-02
  2.6510e-01  1.2582e-01 -3.9834e-01 -2.7832e-01  1.6225e-01  4.5138e-02
 -5.4792e-02 -6.0437e-02 -3.2641e-01 -2.1744e-01 -8.7771e-02  1.3443e-01
  1.3209e-01 -1.0430e-01  1.8851e-01  1.5267e-01  2.5451e-01 -3.8108e-01
  5.0737e-01  1.2683e-01  3.6272e-02 -2.0192e-01 -7.0588e-02 -8.8823e-02
  1.4248e-02 -1.3948e-01 -5.3001e-02  3.8400e-01 -7.1702e-02 -2.8495e-01
  3.9161e-01  6.6414e-01 -1.2302e-01 -3.1577e-01  8.6273e-02  1.6665e-02
 -1.0768e-01 -1.5478e-01 -5.7639e-02 -3.9881e-01  3.4402e-01 -1.6982e-01
  1.4718e-01 -6.1504e-01 -7.8528e-02 -5.3038e-02 -1.4703e-01  1.4217e-01
  5.5486e-01  3.8348e-01  1.3249e-01 -2.2455e-01  

Emb Layer에서 Pre-trained Word Embedding 사용

In [0]:
vocab['일본', '수출']

[4, 16]

In [0]:
input_dim, output_dim = vocab.embedding.idx_to_vec.shape

In [0]:
layer = gluon.nn.Embedding(input_dim, output_dim)
layer.initialize(ctx=mx.gpu())
layer.weight.set_data(vocab.embedding.idx_to_vec)

In [0]:
emb_out = layer(nd.array([4, 16], ctx = mx.gpu()))

In [0]:
emb_out


[[ 1.8810e-01  1.9917e-01 -3.8932e-01  8.2046e-02  4.6638e-02  4.5055e-02
  -3.1671e-01 -3.6490e-02  5.0224e-02 -1.3456e-01  6.7149e-02  3.5040e-01
  -3.7936e-02  1.1197e-01 -4.0459e-01  3.6445e-01  3.8595e-01  1.4581e-01
  -1.4990e-01 -2.9435e-01  2.5718e-01 -6.7448e-02  2.2115e-01 -4.3913e-01
   7.7741e-02 -1.7991e-01 -3.2989e-01  1.1673e-01 -3.1833e-01 -1.6808e-02
  -2.8415e-02  3.6290e-02  2.6130e-01 -4.2483e-01 -2.7343e-01  4.2707e-01
  -7.9224e-02  4.2217e-02 -2.5600e-01 -4.6470e-04 -3.3134e-02  4.3725e-02
   1.7770e-01 -1.4922e-01  3.9644e-01 -1.2606e-01 -2.5324e-02 -9.2616e-02
   1.5082e-02  1.5065e-01  2.9300e-01  3.6282e-01 -1.0867e-01  8.3840e-02
   4.4520e-02 -6.1860e-02 -3.8661e-01 -7.9247e-02 -7.2375e-02 -5.9540e-02
   2.4200e-01 -1.1036e-01  2.0190e-03  1.2865e-01 -6.6143e-02  3.5190e-02
   2.2175e-01 -1.3672e-01  1.7555e-01 -2.4323e-01  2.6840e-01  1.9171e-01
  -1.1305e-01 -1.6910e-01 -3.9212e-03 -2.0387e-02 -3.6294e-01  1.1597e-01
   4.2091e-02 -5.8388e-02 -9.0254e-02

 기존에 만들어진 Vocab 활용

In [0]:
vocab_pretrained = nlp.Vocab(nlp.data.Counter(fasttext_simple.idx_to_token))

In [0]:
len(vocab_pretrained)

879133

In [0]:
vocab_pretrained.token_to_idx['수출']

472039

In [0]:
vocab_pretrained.idx_to_token[591821]

'위치였기'

![대체 텍스트](https://)


Word Similarity

In [0]:
def norm_vecs_by_row(x):
    return x / nd.sqrt(nd.sum(x * x, axis=1) + 1E-10).reshape((-1,1))


In [0]:
def get_knn(vocab, k, word):
    word_vec = vocab.embedding[word].reshape((-1, 1))
    vocab_vecs = norm_vecs_by_row(vocab.embedding.idx_to_vec)
    dot_prod = nd.dot(vocab_vecs, word_vec)
    indices = nd.topk(dot_prod.reshape((len(vocab), )), k=k+1, ret_typ='indices')
    indices = [int(i.asscalar()) for i in indices]
    # Remove input tokens.
    return vocab.to_tokens(indices[1:])

In [0]:
def cos_sim(x, y):
    return nd.dot(x, y) / (nd.norm(x) * nd.norm(y))

In [0]:
get_knn(vocab, 5, '수출')

['무인양품', '규제', '브랜드의', '불매운동', '앱']

In [0]:
vocab_pretrained.set_embedding(fasttext_simple)

In [0]:
get_knn(vocab_pretrained, 5, '한국')

['한국뿐', '한국와', '한국뿐만', '대한민국뿐만', '대한민국와']

In [0]:
cos_sim(vocab_pretrained.embedding['한국'], vocab_pretrained.embedding['일본'])


[0.58051175]
<NDArray 1 @cpu(0)>